In [1]:
from itertools import permutations
import numpy as np
from enum import Enum
from sortedcontainers import SortedList

In [14]:
class StartType(Enum):
    POETRY = 0
    START_OF_SENTENCE = 1
    MID_SENTENCE = 2
    AMBIGUOUS = 3

class EndType(Enum):
    NO_END = 0
    MAYBE_END = 1
    LIKELY_END = 2

class PictureType(Enum):
    UNCATEGORIZED = 0
    CLOCK = 1
    MAP = 2
    PERSON_W_CARD = 3
    BW_ETCHING = 4
    WHITE_LINES = 5
    BLACK_LINES = 6
    PERSON_WEAPON = 7
    STAIN_GLASS = 8
    GALLERIE = 9
    SCULPTURE = 10

In [3]:
class Card:
    def __init__(self, number):#, most_likely_to_follow={}, less_likely_to_follow={}):
        self.number = number
        self.picture_type = PictureType.UNCATEGORIZED
        self.next_card_likelihood_scores = np.zeros(100)
        self.start_type = StartType.MID_SENTENCE
        self.end_completes_sentence = EndType.NO_END
        self.poetry = False
        self.time = False
        self.they_them = False
        self.bible_verse = False
        self.mandy = False
        self.moles = False
        # self.most_likely_to_follow = most_likely_to_follow
        # self.less_likely_to_follow = less_likely_to_follow
        # self.has_poem = False
    # def push_most_likely_to_follow(self, card_set):
    #     self.most_likely_to_follow.update(card_set)
    # def push_less_likely_to_follow(self, card_set):
    #     self.less_likely_to_follow.update(card_set)
    def set_likelihood_scores(self, card_nums, score):
        self.next_card_likelihood_scores[card_nums] = score

In [4]:
class Order:
    def __init__(self, order, likelihood=0):
        self.order = order
        self.likelihood = likelihood

In [17]:
class Person:
    def __init__(self, name, mentions=[], murderer=False, murdered=False):
        self.name = name
        self.murderer = murderer
        self.murdered = murdered
        self.mentions = mentions
    def add_mention(self, card_nums):
        self.mentions.update(card_nums)

In [5]:
# def ordering_likelihood(order, cards):
#     likelihood_score = 0
#     for i in range(len(order)-1):
#         card_num = order[i]
#         if order[i+1] in cards[card_num].most_likely_to_follow:
#             likelihood_score += 2
#         elif order[i+1] in cards[card_num].less_likely_to_follow:
#             likelihood_score += 1
#     return likelihood_score

# def ordering_likelihood_faster(order, cards):
#     likelihood_score = 0
#     for i in range(len(order)-1):
#         card_num = order[i]
#         if order[i+1] in cards[card_num].most_likely_to_follow:
#             likelihood_score += 2
#         elif order[i+1] in cards[card_num].less_likely_to_follow:
#             likelihood_score += 1
#         else:
#             return 0
#     return likelihood_score
        

In [6]:
def ordering_likelihood(order, cards):
    likelihood_score = 0
    for i in range(len(order)-1):
        likelihood_score += cards[order[i]].next_card_likelihood_scores[order[i+1]]
    return likelihood_score

def ordering_likelihood_faster(order, cards):
    likelihood_metric = 0
    for i in range(len(order)-1):
        next_score = cards[order[i]].next_card_likelihood_scores[order[i+1]]
        if next_score == 0:
            return 0
        likelihood_metric += next_score
    return likelihood_metric

In [22]:
def most_likely_orders(cards, num_top_orders=20):
    # top_orders = [None for i in range(num_top_orders)] #make this sorted list?
    top_orders = {}
    for order in permutations(range(100)):
        likelihood_score = ordering_likelihood_faster(order, cards)
        if likelihood_score != 0:
            if len(top_orders.keys()) < num_top_orders:
                top_orders[likelihood_score] = order
            elif likelihood_score > min(top_orders.keys()):
                top_orders.remove(min(top_orders.keys()))
                top_orders[likelihood_score] = order
        # Order(order, likelihood_score)
        #add to top_orders and it will sort itself
        #remove last item from top_orders caues it's the lowest likelihood
    return top_orders

In [8]:
cards = np.array([Card(i) for i in range(100)])

In [ ]:
people = np.array([Person("bob carrothers", mentions=[79]),
                   Person("tom broom", mentions=[85, 86, 94, 67]),
                   Person("tom brown", mentions=[77]),
                   Person("billy moon", mentions=[74]),
                   Person("bill somerset", mentions=[90]),
                   Person("isabel acher", mentions=[99]),
                   Person("jimmy hales", mentions=[51]),
                   Person("calloo callay", mentions=[71]),
                   Person("agnes black", mentions=[73]),
                   Person("santa sede", mentions=[95]),
                   Person("stefanos", mentions=[75], murderer=True),
                   Person("sasha", mentions=[75,77], murderer=True),
                   Person("duncan flanders", mentions=[13,16, 55]), #55 is maybe
                   Person("alfredo catelli", mentions=[42]),
                   Person("diamond jim", mentions=[43]),
                   Person("orazio the butcher", mentions=[75], murderer=True),
                   Person("ethelbald caligula", mentions=[52]),
                   Person("peter kendrick", mentions=[9]),
                   Person("zoe green", mentions=[71,74,28]),
                   Person("eve sullivan mcqueen", mentions=[57], murderer=True),
                   Person("iris murdock", mentions=[62]),
                   Person("captain webb", mentions=[3,11,30,54,71]),
                   Person("sarah e.c. byrne", mentions=[12]),
                   Person("ermintrude clotterbuck", mentions=[12]),
                   Person("mandy", mentions=[15, 21, 27, 56, 96]),
                   Person("angie duncan", mentions=[20, 55]),#55 is maybe
                   Person("andy richards", mentions=[24]),
                   Person("tom the junkie", mentions=[27]),
                   Person("april of dunstable", mentions=[40]),
                   Person("meg", mentions=[21, 68, 16]), #16 not by name
                   Person("bailey", mentions=[3, 53, 51, 99, 9, 53]),
                   Person("stephen", mentions=[17, 82]),
                   Person("sam", mentions=[99, 88, 10, 53, 55]), # 53 is maybe
                   Person("maud", mentions=[69], murderer=True),
                   Person("person who hangs with cerberus", mentions=[], murderer=True),
                   Person("armand", mentions=[65], murdered=True),
                   Person("urquhart", mentions=[20,85]),
                   Person("horace", mentions=[86,24,98,44,25], murdered=True),
                   Person("dean", mentions=[89, 52]),
                   Person("gilbert king", mentions=[13])
                ])

# "hero" used a katana - 76, guy with headwrap
# arthur used a gun - 2
# danny used a bottle - 43

In [9]:
## Grammatical characteristics 
sentence_starts = [0,9,10,13,14,16,18,38,41,42,43,44,45,46,98,29,49,50,51,52]
ambiguous = [33,47,48,18,28,93]
for i in sentence_starts:
    cards[i].start_type = StartType.START_OF_SENTENCE
for i in ambiguous:
    cards[i].start_type = StartType.AMBIGUOUS
    
maybe_ends_sentence = [35, 74, 96, 97, 87, 22, 46, 4, 7, 29, 49, 32, 57, 96, 14, 44]
likely_ends_sentence = [57, 27, 31, 55, 56, 13, 98, 89, 60, 58, 48, 1, 17, 21, 27, 53, 54]
for i in maybe_ends_sentence:
    cards[i].end_completes_sentence = EndType.MAYBE_END
for i in likely_ends_sentence:
    cards[i].end_completes_sentence = EndType.LIKELY_END
    
poetry_likely_to_follow = [50]
name_should_follow = []


In [10]:
## Thematic characteristics in text
poetry = [29,49,50,51,52]
for i in poetry:
    cards[i].poetry = True
times = [17,51]
for i in times:
    cards[i].time = True
they_them = []
for i in poetry:
    cards[i].they_them = True
bible_verses = []
for i in bible_verses:
    cards[i].bible_verse = True
mandy = []
for i in mandy:
    cards[i].mandy = True
moles = []
for i in moles:
    cards[i].moles = True

In [12]:
## Picture categories
pic_categorization = {
    PictureType.UNCATEGORIZED : [72],
    PictureType.CLOCK : [10, 1, 11, 32, 37, 61, 62, 67, 28, 48], #10
    PictureType.MAP : [2, 56, 58, 75, 78, 84, 92, 43, 49, 33], #10
    PictureType.PERSON_W_CARD : [3, 12, 23, 57, 59, 97, 71, 69, 16, 50], #10
    PictureType.BW_ETCHING : [4, 7, 22, 34, 53, 70, 66, 51, 17, 80], #10
    PictureType.WHITE_LINES : [5, 55, 64, 73, 87, 83, 88, 14, 0], #9
    PictureType.BLACK_LINES : [31, 99,  44, 8, 98, 93, 27, 60, 20, 74], #10
    PictureType.PERSON_WEAPON : [9, 21, 19, 24, 36, 40, 90, 63, 52, 76], #10
    PictureType.STAIN_GLASS : [54, 65, 68, 77, 81, 91, 96, 18, 41, 15], #10
    PictureType.GALLERIE : [85, 82, 79, 95, 94, 25, 6, 13, 38, 42], #10
    PictureType.SCULPTURE : [30, 39, 89, 26, 35, 45, 46, 29, 47, 86] #10
}
for pic_type, card_nums in pic_categorization.items():
    for i in card_nums:
        cards[i].picture_type = pic_type

In [15]:
## Animals
stain_glass_animal_letters = {
    "rhino" : ["o","e","s"],
    "dog": ["c","d"],
    "wolf": ["b"],
    "porpoise":["o"],
    "bull":["v"],
    "elephant":["f","a","t"],
    "panda":["f"],
    "polar bear":["j"],
    "ostrich":["u"],
    "hippo": ["n"],
    "hawk":["xyz","m"],
    "tapir": ["g"],
    "kangaroo": ["k"],
    "alligator": ["l"],
    "bear": ["w"],
    "swan": ["i"],
    "boar": ["h","r"]
}
animal_mentions = {
    "rhino" : [],
    "dog": [],
    "wolf": [65, 21],
    "porpoise":[92],
    "bull":[76],
    "elephant":[],
    "panda":[],
    "polar bear":[],
    "ostrich":[],
    "hippo": [],
    "hawk":[],
    "tapir": [],
    "kangaroo": [],
    "alligator": [],
    "bear": [53],
    "swan": [],
    "boar": [80]
}

In [23]:
print(most_likely_orders(cards, num_top_orders=2))

KeyboardInterrupt: 

In [ ]:
## Murderers with weapons
"""
genie/smoke/on couch; ??
bejeweled old woman clutching man; pistol
royal; axe
headscarf woman; cup (poison?)
detective; oven
man with cat, striped red hat, scarf; arrow gun
balding man; pistol with laser
big nose man with suit, bow, necklace, drink; surgical tools
headwrapped man with facial hair and bushy eyebrows; katana
smiling/winking man; hands/stranglation
"""

In [ ]:
## Gallerie (all paintings are in the Louvre)
"""
6: seaport at sunset by claude lorrain
42: mona lisa by leonardo davinci
94: francois ler, roi de france by tiziano
13: portrait of suzanne fourment
25: the angel leaving tobias and his family by rembrandt
38: seaport by moonlight by claude joseph vernet
95: landscape with shepherds and herd in a forest path by cornelis huysman
79: venus asks vulcan to forge arms for her son aeneas by anthony van dyck
85: landscape with diogenes by nicolas poussin
82: christ carrying the cross by eustache le sueur
"""

In [1]:
## Statue
# Hypothesis: pointing to murderers, if there are two then murderer killed another murderer
"""
35: Zeus or Poseidon from Artemision, 5h cent BC, in National Archaeological Museum Athens, photoshopped into library
46: Dendera temple of hathor, egypt, middle and right characters are switched
26: uranos und der tanz der gestirne, in rauch-schinkel-mueum in berlin (copy in treppenhalle des alten museums du berlin?), lat/long photoshopped on top
29: gallery of daily life in classical antiquity
"""

'\n35: Zeus or Poseidon from Artemision, 5h cent BC, in National Archaeological Museum Athens, photoshopped into library\n46: Dendera temple of hathor, egypt, middle and right characters are switched\n26: uranos und der tanz der gestirne, in rauch-schinkel-mueum in berlin (copy in treppenhalle des alten museums du berlin?), lat/long photoshopped on top\n29: gallery of daily life in classical antiquity\n'

In [ ]:
## White lines/dates
"""
0: elephant sculpture; jan 4 (13)

87: church, possibly or the holy sepulchre but columns don't look quite right; apr 4 (10)
64: tiled mural at the general havelock pub in hastings; sept 4 (4)

73: fontana delle anfore in rome; march 4 (8)
5: harbor; aug 4 (5)
88: museum/library; may 6 (6)

83: old building; june 6 (8)
14: grand central station in nyc; july 7 (7)
55: gdansk harbor in poland; feb 5 (6)
"""

In [ ]:
## People with cards equation


In [ ]:
number = 8
less_likely_to_follow = {1, 2}
most_likely_to_follow = {3, 4, 5, 6, 7}
unlikely_to_follow = range(9, 40+1)
cards[number].set_likelihood_scores(most_likely_to_follow, 2)
cards[number].set_likelihood_scores(less_likely_to_follow, 1)
# cards[number].push_most_likely_to_follow(most_likely_to_follow)
# cards[number].push_less_likely_to_follow(less_likely_to_follow)